Prepare Main project

In [1]:
use_google_colab = False
first_run = True

In [2]:
# Clone main project
if(use_google_colab and first_run):
    print("clone main project")
    !git clone "https://github.com/albirrkarim/post-processing-segment-anything-model.git"
    !cd /content/post-processing-segment-anything-model && mv * "../"
    !rm -rf "post-processing-segment-anything-model"

In [3]:
if(use_google_colab and first_run):
    print("Prepare folder")
    !mkdir precompute
    !mkdir output
    from google.colab import drive
    drive.mount('/content/drive')


Prepare Algorithms

In [4]:
# Clone all algorithm sam, matteformer, aim, inspyrenet

if(use_google_colab and first_run):
    print("clone all algorithm sam, matteformer, aim, inspyrenet")

    # !cd "algorithms" && git clone https://github.com/webtoon/matteformer.git

    !cd "algorithms" && git clone https://github.com/facebookresearch/segment-anything.git

    # !cd "algorithms" && git clone https://github.com/JizhiziLi/AIM.git

    !pip install transparent-background

    !pip install "algorithms/segment-anything"

    !pip install pymatting

Prepare Models (.pth)

In [5]:
if(use_google_colab):
    MODEL_DIR = "models/"
    AIM_NET_MODEL_PATH = MODEL_DIR+"aimnet_pretrained_matting.pth"
    RESNET_MODEL_PATH = MODEL_DIR+"r34mp_pretrained_imagenet.pth.tar"
    MATTEFORMER_MODEL_PATH = MODEL_DIR+"matteformer.pth"
    INSPYRENET_MODEL_PATH = MODEL_DIR+"InSPyReNet_SwinB_DIS5K.pth"
else:
    MODEL_DIR = "/Users/susanto/Documents/Proyek/best-remove-background/models/"
    AIM_NET_MODEL_PATH = MODEL_DIR+"aimnet_pretrained_matting.pth"
    RESNET_MODEL_PATH = MODEL_DIR+"r34mp_pretrained_imagenet.pth.tar"
    MATTEFORMER_MODEL_PATH = MODEL_DIR+"matteformer_image_matting.pth"
    INSPYRENET_MODEL_PATH = MODEL_DIR+"InSPyReNet_SwinB_Large.pth"

In [6]:
# download all models .pth
import gdown

if (use_google_colab and first_run):
    print("Download all models")
    !mkdir "models"

    gdown.download(
        "https://drive.google.com/uc?id=1XBa2oc0yW1WTKOJ0ECr_EK-qIQj4rXIt&export=download",
        output="datasets/aim-500.zip")
    
    !unzip -q "datasets/aim-500.zip" -d "datasets"

    # AIM NET
    # aimnet_pretrained_matting.pth
    gdown.download(
        "https://drive.google.com/uc?export=download&id=16dd1FGMcsMTqR6EfD2T9mtRmPwxnY0zs",
        output=AIM_NET_MODEL_PATH)

    # r34mp_pretrained_imagenet.pth.tar
    gdown.download("https://drive.google.com/uc?export=download&id=18Pt-klsbkiyonMdGi6dytExQEjzBnHwY",
                   output=RESNET_MODEL_PATH)

    # Matteformer
    gdown.download("https://drive.google.com/u/0/uc?id=1AU7uM1dtYjEhtOa_9OGfoQUE-tmW9mX5&export=download",
                   output=MATTEFORMER_MODEL_PATH)

    # Inspyrenet
    gdown.download("https://drive.google.com/u/0/uc?id=1aCxHMbhvj8ah77jXVgqvqImQA_Y0G-Yg&export=download",
                   output=INSPYRENET_MODEL_PATH)

    first_run = False


# Import all make model

In [7]:
import torch
import sys
sys.path.append('driver')
sys.path.append('algorithms/AIM/core')

In [8]:
DEVICE = "cpu"

In [9]:
# AIM
from aim import makeAIMNetModel, inference_img as AIMNET_Predictor
from aim import calculate_sad_mse_mad_whole_img,compute_connectivity_loss_whole_image,compute_gradient_whole_image

aim_model = makeAIMNetModel(device=DEVICE,model_path=AIM_NET_MODEL_PATH,res_net_model_path=RESNET_MODEL_PATH)

In [10]:
# Inspyrenet

from transparent_background import Remover

InSPyReNet_remover = Remover(
    fast=True, jit=True, device=DEVICE, ckpt=INSPYRENET_MODEL_PATH)


B /Users/susanto/Documents/Proyek/best-remove-background/models/InSPyReNet_SwinB_Large.pth
Settings -> Mode=fast, Device=cpu, Torchscript=enabled


In [11]:
# Matteformer
sys.path.append('algorithms/matteformer')

import networks as matteformer_networks
import utils_matteformer as utils
from inference import generator_tensor_dict, single_inference


def makeMatteFormerModel(device='cpu'):
    checkpoint_path = MATTEFORMER_MODEL_PATH

    # build model
    model = matteformer_networks.get_generator(is_train=False)
    # model.cpu()

    # load checkpoint
    checkpoint = torch.load(checkpoint_path, map_location=torch.device(device))

    model.load_state_dict(utils.remove_prefix_state_dict(
        checkpoint['state_dict']), strict=True)

    model.to(device)
    model = model.eval()

    return model

matteformer_model = makeMatteFormerModel(DEVICE)

def ImageMattingMatteFormer(image_path, trimap_path):
    image_dict = generator_tensor_dict(image_path, trimap_path)
    alpha_pred = single_inference(matteformer_model, image_dict, device=DEVICE)

    return alpha_pred

In [12]:
sys.path.append('helper')

from helper import *

from post_helper import *

from sam_helper import *

from evaluation_helper import *

import os
import time
import cv2
import numpy as np
import pickle
import datetime


OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


# Defining function 

In [13]:
def rgbToAlpha2D(rgb_gray_scale):
    # # Convert the image to grayscale
    # gray_image = np.mean(rgb_gray_scale, axis=2)

    # # Normalize the pixel values between 0 and 1
    # normalized_image = gray_image / np.max(gray_image)

    # # Scale the values to the desired range (0-255)
    # scaled_image = (normalized_image * 255).astype(np.uint8)

    # return scaled_image
    alpha = np.array(rgb_gray_scale)
    alpha = alpha[:, :, 0] if alpha.ndim > 2 else alpha

    return alpha


In [14]:
def getMSEandSAD(rgba_img, alpha_img):
    # mse = getMSE(rgba_img, alpha_img)
    # sad = getSAD(rgba_img, alpha_img)

    predict = rgba_img[:, :, 3]

    predict = predict / 255
    alpha_img = alpha_img / 255

    sad, mse, mad = calculate_sad_mse_mad_whole_img(predict, alpha_img)
    conn = compute_connectivity_loss_whole_image(predict, alpha_img)
    grad = compute_gradient_whole_image(predict, alpha_img)

    return {
        "sad": sad,
        "mse": mse,
        "mad": mad,
        "conn": conn,
        "grad": grad
    }


In [15]:
def crop_image_with_bounding_box(img, bounding_box):
    x1, y1, x2, y2 = bounding_box
    cropped_image = img[y1:y2, x1:x2,]
    return cropped_image


In [16]:
def getImage(name, folder):
    return np.asarray(Image.open("datasets/AIM-500/"+folder+"/"+name))

In [17]:
def cropUnusedBlankPixelExtended(myImage):
    # https://stackoverflow.com/a/53829086
    if isinstance(myImage, Image.Image):
        PIL_image = myImage
    else:
        PIL_image = Image.fromarray(myImage.astype('uint8'), 'RGBA')

    bounding = PIL_image.getbbox()

    bounding = extendBoundaries(np.asarray(myImage), bounding, 50)

    if (bounding != None):
        PIL_image = PIL_image.crop(bounding)
    else:
        width, height = PIL_image.size
        bounding = (0, 0, width, height)

    return [PIL_image, np.array(bounding).astype(int)]


In [18]:
def addTheRestMask(masks):
    mask = masks[0]['segmentation']

    for i in range(0, len(masks)):
        # Apply Segment Mask
        segmentation = masks[i]['segmentation']
        mask = np.logical_or(mask, segmentation)

    inverted_mask = mask ^ True

    masks.append({
        "segmentation": inverted_mask
    })

    return masks


In [19]:
def findSamObjectMaskThatMatchDataset(file_name, masks, threshold=0.5):
    # if the iou between sam and the original is < 0.5
    # it mean sam detect false object from the original object in dataset.
    # then don't do post processing because the dataset AIM-500 is focused on one object in the picture

    file_name_origin = os.path.splitext(file_name)[0]
    maximal = 0
    maximal_index = 0

    # Evaluate ori vs SAM
    origin_img = getImage(file_name_origin+".png", "mask")
    origin_mask = origin_img[:, :,
                             0] > 0 if origin_img.ndim > 2 else origin_img > 0

    for index in range(0, len(masks)):
        sam_mask = masks[index]["segmentation"]

        # Evaluate SAM before post processing
        iou = getIoU(origin_mask, sam_mask)

        if (iou > maximal):
            maximal = iou
            maximal_index = index

    # add more mask if the iou is increased
    basic_mask = masks[maximal_index]["segmentation"]
    current_iou = maximal

    for index in range(0, len(masks)):
        sam_mask = masks[index]["segmentation"]

        result = np.logical_or(sam_mask, basic_mask)

        # Evaluate SAM before post processing
        iou = getIoU(origin_mask, result)

        if (iou > current_iou):
            current_iou = iou
            basic_mask = result

    return basic_mask


# Processing pipeline

In [20]:
CACHE=False

In [21]:
def useCache(IDX_IMG, pipeline_name, value=None):
    if (not CACHE):
        return None

    cache_path = "precompute/"+IDX_IMG+"_"+pipeline_name+".pickle"
    if (value == None):
        if (os.path.exists(cache_path)):
            # Open the pickle file
            with open(cache_path, "rb") as file:
                loaded_dict = pickle.load(file)
            return loaded_dict
        return None
    else:
        # save cache
        # Save the dictionary to the generated filename
        with open(cache_path, "wb") as file:
            pickle.dump(value, file)

    return None


In [22]:
def AIM_Pipeline(IDX_IMG, PIL_image, dilated_sam_mask, croped, blank_image, bounding, origin_alpha, origin_mask):
    cache = useCache(IDX_IMG, "aim")

    if (cache == None):
        start_time = time.time()

        predict = AIMNET_Predictor(aim_model,np.asarray(PIL_image),device=DEVICE)

        predict = (predict * 255).astype(np.uint8)

        predict[dilated_sam_mask == False] = 0

        applied_alpha_aim = croped.copy()
        applied_alpha_aim[:, :, 3] = predict

        # back to original size
        a = blank_image.copy()
        alpha_aim_original_size = patchToBoundingBox(
            a, bounding, applied_alpha_aim)

        Image.fromarray(alpha_aim_original_size).save(
            IDX_IMG+"_aim.png")

        # Evaluate SAM before post processing
        AIM_result_origin_size_mask = alpha_aim_original_size[:, :, 3] > 0

        res = getMSEandSAD(alpha_aim_original_size, origin_alpha)
        res["iou"] = getIoU(origin_mask, AIM_result_origin_size_mask)
        res["time"] = time.time() - start_time

        useCache(IDX_IMG, "aim", [predict, res])
    else:
        predict, res = cache

    return predict, res


In [23]:
def Matteformer_Pipeline(IDX_IMG, name_before, JPG_PATH, croped, blank_image, bounding, origin_alpha, origin_mask):
    cache = useCache(IDX_IMG, "matteformer_"+name_before)

    if (cache == None):
        start_time = time.time()

        # Image Matting: MatteFormer
        alpha_pred = ImageMattingMatteFormer(JPG_PATH, IDX_IMG+"_trimap_"+name_before+".jpg")

        croped_a = croped.copy()
        croped_a[:, :, 3] = alpha_pred

        # Back to original size
        a = blank_image.copy()
        croped_a = patchToBoundingBox(a, bounding, croped_a)

        Image.fromarray(croped_a).save(
            IDX_IMG+"_"+name_before+"_matteformer.png")

        MatteFormer_result_mask = croped_a[:, :, 3] > 0

        res = getMSEandSAD(croped_a, origin_alpha)
        res["iou"] = getIoU(origin_mask, MatteFormer_result_mask)
        res["time"] = time.time() - start_time

        useCache(IDX_IMG, "matteformer_"+name_before, res)
    else:
        res = cache

    return res


In [24]:
def InSpyReNet_Pipeline(IDX_IMG, JPG_PATH, dilated_sam_mask, blank_image, bounding, origin_alpha, origin_mask):
    cache = useCache(IDX_IMG, "InSpyReNet")

    if (cache == None):

        start_time = time.time()

        PIL_image = Image.open(JPG_PATH)
        InSpyReNet_result = InSPyReNet_remover.process(PIL_image)

        # Remove all region  outside dilated mask
        InSpyReNet_result[dilated_sam_mask == False] = [0, 0, 0, 0]

        # Back to original size
        a = blank_image.copy()
        InSpyReNet_result_origin_size = patchToBoundingBox(
            a, bounding, InSpyReNet_result)

        Image.fromarray(InSpyReNet_result_origin_size).save(
            IDX_IMG+"_InSpy.png")

        InSpyReNet_result_origin_size_mask = InSpyReNet_result_origin_size[:, :, 3] > 0

        res = getMSEandSAD(InSpyReNet_result_origin_size, origin_alpha)
        res["iou"] = getIoU(
            origin_mask, InSpyReNet_result_origin_size_mask)

        res["time"] = time.time() - start_time

        useCache(IDX_IMG, "InSpyReNet", [InSpyReNet_result, res])
    else:
        InSpyReNet_result, res = cache

    return InSpyReNet_result, res


In [25]:
def makeTrimapPlusAIM(img, aim_trimap, s_size=2, name="example"):
    # Step 1: Select Only Main Visible Area
    main_area_mask = img[:, :, 3] > 170
    # plt.imshow(main_area_mask)

    # Step 2: Detect Edges (For nets rope, etc)
    # limit edge detection to visible area only
    visible_area = img.copy()
    visible_area[img[:, :, 3] < 200] = [0, 0, 0, 0]
    PIL_image, bounding = cropUnusedBlankPixel(visible_area)
    x, y, x1, y1 = bounding
    croped_img = img[y:y1, x:x1]

    # Convert RGBA image to grayscale
    gray_image = cv2.cvtColor(
        croped_img.astype(np.uint8), cv2.COLOR_RGBA2GRAY)

    # Apply Canny edge detection
    edges = cv2.Canny(gray_image, threshold1=100, threshold2=200)
    threshold = 0  # Set the threshold value
    edges_mask = edges > threshold

    # Back to original size
    blank_img = np.zeros((img.shape[0], img.shape[1]), dtype=np.uint8)
    blank_img = blank_img != 0
    edges_mask = patchToBoundingBox(blank_img, bounding, edges_mask)

    # Remove outside visible_area
    visible_area = img[:, :, 3] > 20
    edges_mask = np.logical_and(edges_mask, visible_area)
    # plt.imshow(edges_mask)

    # Step 3: Perform Closing Into Edge region
    dilated = binary_dilation(edges_mask, structure=np.ones((s_size, s_size)))
    closed_edge_mask = binary_erosion(
        dilated, structure=np.ones((s_size, s_size)))

    # plt.imshow(dilated)

    # Step 4: main_area_mask + closed_edge_mask
    fore_mask = np.logical_or(main_area_mask, closed_edge_mask)

    # Step 5: Get Transition Mask
    erosion_fore_mask = binary_erosion(
        fore_mask, structure=np.ones((s_size, s_size)))
    dilated_fore_mask = binary_dilation(
        fore_mask, structure=np.ones((s_size, s_size)))
    transition_mask = np.logical_and(
        dilated_fore_mask, np.logical_not(erosion_fore_mask))

    # plt.imshow(fore_mask)

    # Add more transition_mask based on closed_edge_mask
    transition_mask = np.logical_or(transition_mask, closed_edge_mask)

    backbone = binary_erosion(
        closed_edge_mask, structure=np.ones((s_size+1, s_size+1)))
    # plt.imshow(backbone)

    absolute_foreground = img[:, :, 3] == 255

    backbone = np.logical_or(absolute_foreground, backbone)

    f = (img[:, :, 3] < 250) & (img[:, :, 3] > 50)

    # Create the trimap
    trimap = np.zeros((img.shape[0], img.shape[1]), dtype=np.uint8)
    trimap[fore_mask] = 255
    trimap[transition_mask] = 127
    trimap[f] = 127
    trimap[backbone] = 255

    trimap[aim_trimap == 0] = 0
    mask1 = aim_trimap == 127
    mask2 = trimap == 0
    mask3 = np.logical_and(mask1, mask2)
    trimap[mask3] = 0

    # saveMask(fore_mask, "fore.jpg")
    # saveMask(transition_mask, "transition_mask.jpg")
    # saveMask(backbone, "backbone.jpg")
    # saveMask(f, "fa.jpg")

    return trimap.astype(np.uint8)


# Evaluations

In [26]:
def testPostProcessing(file_name, sam_mask):

    # Extract the base name without the extension
    file_name_origin = os.path.splitext(file_name)[0]

    image_path = 'datasets/AIM-500/original/'+file_name_origin+".jpg"
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # RGB numpy array to RGBA numpy array
    rgba_image = np.zeros((image.shape[0], image.shape[1], 4), dtype=np.uint8)
    rgba_image[:, :, :3] = image
    rgba_image[:, :, 3] = 255

    blank_image = np.zeros(
        (rgba_image.shape[0], rgba_image.shape[1], 4), dtype=np.uint8)

    OUTPUT_FOLDER = "output/"

    index = file_name_origin

    evaluation = {
        "sam": 0,

        "InSpyReNet": 0,
        "InSpyReNet_MatteFormer": 0,

        "AIM": 0,
        "AIM_MatteFormer": 0,
    }

    # sam_mask = masks[index]["segmentation"]

    IDX_IMG = OUTPUT_FOLDER+str(index)

    applied_mask = rgba_image.copy()
    applied_mask[sam_mask == False] = [0, 0, 0, 0]

    # Crop aplied transparent region (aplied mask)
    [PIL_image, bounding] = cropUnusedBlankPixelExtended(
        Image.fromarray(applied_mask))

    # Crop original image
    croped = crop_image_with_bounding_box(rgba_image.copy(), bounding)

    # Evaluate ori vs SAM
    origin_img = getImage(file_name_origin+".png", "mask")
    origin_alpha = rgbToAlpha2D(origin_img)
    origin_mask = origin_img[:, :,
                             0] > 0 if origin_img.ndim > 2 else origin_img > 0

    # Save origin image
    origin_dataset = rgba_image.copy()
    origin_dataset[:, :, 3] = origin_alpha

    # Evaluate SAM before post processing
    res = getMSEandSAD(applied_mask, origin_alpha)
    res["iou"] = getIoU(origin_mask, sam_mask)
    evaluation["sam"] = res

    Image.fromarray(origin_dataset).save(IDX_IMG+"_origin_dataset.png")
    PIL_image = Image.fromarray(applied_mask)
    PIL_image.save(IDX_IMG+"_origin_sam_applied.png")
    # PIL_image.save(IDX_IMG+"_origin_mask.png")

    # Save original mask
    saveMask(sam_mask, IDX_IMG+"_sam.jpg")

    # Croped original image to JPG for input AIM and  InSpyReNet
    JPG_PATH = IDX_IMG+"_origin.jpg"
    PIL_image = Image.fromarray(croped)
    PIL_image = PIL_image.convert("RGB")
    PIL_image.save(JPG_PATH)

    # POST PROCESSING PIPELINE
    # Remove all region  outside dilated mask
    cropped_mask = cropToBoundingBox(sam_mask, bounding)
    dilated_sam_mask = binary_dilation(cropped_mask)

    # AIM NET _________________________________________
    predict, res = AIM_Pipeline(
        IDX_IMG, PIL_image, dilated_sam_mask, croped, blank_image, bounding, origin_alpha, origin_mask)

    evaluation["AIM"] = res

    if (not os.path.exists(IDX_IMG+"_trimap_aim.jpg")):
        trimap = np.zeros_like(predict)
        trimap[predict == 0] = 0  # Background
        trimap[predict == 255] = 255  # Foreground
        trimap[(predict > 0) & (predict < 255)] = 128  # Unknown region

        Image.fromarray(trimap).save(IDX_IMG+"_trimap_aim.jpg")
    else:
        trimap = Image.open(IDX_IMG+"_trimap_aim.jpg")
        trimap = np.asarray(trimap)

    res = Matteformer_Pipeline(
        IDX_IMG, "aim", JPG_PATH, croped, blank_image, bounding, origin_alpha, origin_mask)

    evaluation["AIM_MatteFormer"] = res

    # Dichotomous Image Segmentation ______________________

    # InSpyReNet
    InSpyReNet_result, res = InSpyReNet_Pipeline(
        IDX_IMG, JPG_PATH, dilated_sam_mask, blank_image, bounding, origin_alpha, origin_mask)

    evaluation["InSpyReNet"] = res

    # Image Matting Process _______________________________

    # Trimap
    if (not os.path.exists(IDX_IMG+"_trimap_InSpyReNet.jpg")):
        t = makeTrimap(InSpyReNet_result)
        Image.fromarray(t).save(IDX_IMG+"_trimap_InSpyReNet.jpg")

    res = Matteformer_Pipeline(
        IDX_IMG, "InSpyReNet", JPG_PATH, croped, blank_image, bounding, origin_alpha, origin_mask)
    evaluation["InSpyReNet_MatteFormer"] = res


    # Trimap
    if (not os.path.exists(IDX_IMG+"_trimap_aim_InSpyReNet.jpg")):
        t = makeTrimapPlusAIM(InSpyReNet_result, trimap)
        Image.fromarray(t).save(IDX_IMG+"_trimap_aim_InSpyReNet.jpg")

    res = Matteformer_Pipeline(
        IDX_IMG, "aim_InSpyReNet", JPG_PATH, croped, blank_image, bounding, origin_alpha, origin_mask)
    evaluation["InSpyReNet_AIM_MatteFormer"] = res

    return evaluation


In [27]:
def formatEvaluation(arrEval):
    for i in range(0, len(arrEval)):
        print("Eval ", i)
        for key, value in arrEval[i].items():
            print(key, end=" | ")

            # Check if variable is an integer
            if isinstance(value, int):
                print("No value")
            else:
                for ev, v in value.items():
                    formatted_number = "{:.3f}".format(v)
                    print(ev + ':', formatted_number, end=" | ")

            print("\n")

        print("\n\n\n")


In [28]:
def saveEvalResult(my_dict):
    # Get the current date, hour, and minute
    current_time = datetime.datetime.now()
    formatted_time = current_time.strftime("%Y-%m-%d_%H-%M")

    # Generate the filename with date, hour, and minute
    filename = f"eval_result_{formatted_time}.pickle"

    # Save the dictionary to the generated filename
    with open("evaluations/"+filename, "wb") as file:
        pickle.dump(my_dict, file)


In [29]:
def openEvalResult(file_name):
    # Open the pickle file
    with open(file_name, "rb") as file:
        loaded_dict = pickle.load(file)

    return loaded_dict


In [30]:
def printEvaluation(sum_dict):
    for key, value in sum_dict.items():
        if isinstance(value, int):
            print("Evaluation from ", value, " data")
        else:
            print(key, end=" | ")
            for ky, val in value.items():
                formatted_number = "{:.3f}".format(sum_dict[key][ky])
                print(ky + ':', formatted_number, end=" | ")

        print("\n")


In [31]:
def formatEvaluationMean(arrEval):
    length = len(arrEval)
    # Get all key
    sum_dict = {
        "length": length
    }

    for i in range(0, 1):
        for key, value in arrEval[i].items():
            sum_dict[key] = {
                "sad": 0,
                "mse": 0,
                "mad": 0,
                "conn": 0,
                "grad": 0,
                "iou": 0,
                "time": 0
            }

    # print(sum_dict)

    for i in range(0, len(arrEval)):
        for key, value in arrEval[i].items():
            # Check if variable is an integer
            if not isinstance(value, int):
                for ev, v in value.items():
                    sum_dict[key][ev] = sum_dict[key][ev]+v

    # print(sum_dict)
    for key, value in sum_dict.items():
        if not isinstance(value, int):
            for ky, val in value.items():
                sum_dict[key][ky] = sum_dict[key][ky]/length

    # print(sum_dict)
    # print("Evaluation from ", length, " data")

    # PRINT
    printEvaluation(sum_dict)

    # saveEvalResult(sum_dict)
    return sum_dict


In [32]:
arrEval = []

In [35]:
ORIGINAL_PATH = "precompute/sam_mask"
img_list = os.listdir(ORIGINAL_PATH)
# print(img_list)

for i in range(0, 5):
    print(i)
    file_name = img_list[i]
    file_name_origin = os.path.splitext(file_name)[0]

    # if (os.path.exists("output/"+file_name_origin+"_origin_dataset.png") == False):
    PIL_image = Image.open(ORIGINAL_PATH+"/"+file_name)

    mask_np = np.asarray(PIL_image)

    mask = mask_np > 0

    res = testPostProcessing(file_name, mask)

    arrEval.append(res)


0


KeyboardInterrupt: 

In [ ]:
# formatEvaluation(arrEval)

Eval  0
sam | sad: 75.281 | mse: 0.030 | mad: 0.036 | conn: 76.616 | grad: 142.766 | iou: 0.895 | 

InSpyReNet | sad: 41.405 | mse: 0.009 | mad: 0.020 | conn: 39.199 | grad: 48.499 | iou: 0.911 | time: 4.238 | 

InSpyReNet_MatteFormer | sad: 15.445 | mse: 0.002 | mad: 0.007 | conn: 13.315 | grad: 14.088 | iou: 0.925 | time: 11.947 | 

AIM | sad: 27.457 | mse: 0.006 | mad: 0.013 | conn: 25.108 | grad: 45.445 | iou: 0.912 | time: 7.719 | 

AIM_MatteFormer | sad: 21.640 | mse: 0.005 | mad: 0.010 | conn: 19.211 | grad: 33.486 | iou: 0.912 | time: 12.325 | 





Eval  1
sam | sad: 42.521 | mse: 0.020 | mad: 0.024 | conn: 43.442 | grad: 111.873 | iou: 0.966 | 

InSpyReNet | sad: 30.727 | mse: 0.008 | mad: 0.018 | conn: 26.157 | grad: 49.494 | iou: 0.975 | time: 3.091 | 

InSpyReNet_MatteFormer | sad: 15.485 | mse: 0.004 | mad: 0.009 | conn: 14.580 | grad: 15.544 | iou: 0.986 | time: 9.703 | 

AIM | sad: 9.626 | mse: 0.002 | mad: 0.006 | conn: 8.762 | grad: 8.057 | iou: 0.991 | time: 4.882 | 

In [ ]:
sum_dict = formatEvaluationMean(arrEval)

# saveEvalResult(sum_dict)


Evaluation from  5  data


sam | sad: 344.093 | mse: 0.209 | mad: 0.217 | conn: 350.269 | grad: 119.635 | iou: 0.938 | time: 0.000 | 

InSpyReNet | sad: 35.569 | mse: 0.009 | mad: 0.020 | conn: 27.437 | grad: 64.319 | iou: 0.752 | time: 3.102 | 

InSpyReNet_MatteFormer | sad: 19.039 | mse: 0.004 | mad: 0.011 | conn: 9.350 | grad: 27.797 | iou: 0.763 | time: 9.733 | 

AIM | sad: 19.828 | mse: 0.004 | mad: 0.011 | conn: 13.443 | grad: 28.477 | iou: 0.769 | time: 5.732 | 

AIM_MatteFormer | sad: 19.086 | mse: 0.004 | mad: 0.011 | conn: 12.546 | grad: 27.499 | iou: 0.763 | time: 9.829 | 

InSpyReNet_AIM_MatteFormer | sad: 19.039 | mse: 0.004 | mad: 0.011 | conn: 9.350 | grad: 27.797 | iou: 0.763 | time: 10.068 | 



In [45]:
a = openEvalResult("evaluations/eval-500_result_2023-06-02_12-34.pickle")

# print(a)
# formatEvaluation(a)
printEvaluation(a)


Evaluation from  500  data


sam | sad: 80.105 | mse: 0.042 | mad: 0.049 | conn: 82.025 | grad: 97.745 | iou: 0.915 | time: 0.000 | 

InSpyReNet | sad: 60.857 | mse: 0.025 | mad: 0.036 | conn: 60.848 | grad: 107.096 | iou: 0.915 | time: 2.430 | 

InSpyReNet_MatteFormer | sad: 39.920 | mse: 0.016 | mad: 0.024 | conn: 39.368 | grad: 53.031 | iou: 0.912 | time: 3.032 | 

AIM | sad: 41.856 | mse: 0.016 | mad: 0.025 | conn: 39.805 | grad: 36.662 | iou: 0.900 | time: 2.983 | 

AIM_MatteFormer | sad: 55.593 | mse: 0.024 | mad: 0.033 | conn: 53.270 | grad: 50.751 | iou: 0.888 | time: 2.971 | 

